### Introduction/ Business Problem
The question I will be addressing is: "In NY city, if someone is looking to open a restaurant, where would you recommend that they open it?" As NY City is a hotspot for businesses, potential restaurant owners might be curious as to where is the best area to rent their new restaurant at. Using the Fourspace data with the trending venues function, we are able to find the locations around an area that are trending so that potential restaurant owners can consider these areas as their future restaurant space.

### Data
I will be using data from Fourspace, using the base location of new york city and using the trending venues function to find the locations around new york city that are currently trending. With this, potential restaurant owners are able to gain insight in trending spots that they can open their restaurants in, so that they can get a boost in their sales at the opening.

#### Import necessary libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


#### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'LLRPULW0DMNIPKQIZNJFJTXMI2E1FCQURAAJRJAIKL1EXNTP' # your Foursquare ID
CLIENT_SECRET = 'FDVMWT2BDODEUQLLERY2AG4MKUV1MIEV4EP0XDBWC20AGDGK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: LLRPULW0DMNIPKQIZNJFJTXMI2E1FCQURAAJRJAIKL1EXNTP
CLIENT_SECRET:FDVMWT2BDODEUQLLERY2AG4MKUV1MIEV4EP0XDBWC20AGDGK


#### Finding New York City's latitude and longitude coordinates.

In [3]:
address = 'New York'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


#### Get the trending venues around New York City center

In [4]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed4d216949393001c9112c2'},
 'response': {'venues': []}}

In [5]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [6]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [7]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [8]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

#### As the trending venues change accordingly to live updates, the current trending venues are mostly unavailable due to the current COVID-19 impact. 
#### In the future, this can be implemented to see trending venues from any location in the world by changing the "address" variable above. 